# COMS 4995 HW4
#### Jianfeng Zhuang: JZ3872
#### Huazhang Liu: HL3338

### Task 3

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from scipy.stats import randint
import spacy
from transformers import pipeline

In [2]:
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_transformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import Ridge

#### Doing the same preprocessing like in Task 1 first

In [3]:
wine = pd.read_csv("winemag-data-130k-v2.csv")
wine = wine.drop("Unnamed: 0", axis = 1)
# wine.head()

In [4]:
wine_US = wine[wine['country'] == "US"]
wine_US = wine_US.drop('country', axis = 1)
wine_US.index = np.arange(0, wine_US.shape[0])
# wine_US

In [5]:
wine_US['points'] = wine_US['points'].astype('float64')
wine_US_1 = wine_US.copy()
wine_US_1.head()

,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,"Tart and snappy, the flavors of lime flesh and...",NaN,87.0,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
1,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87.0,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
2,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87.0,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
3,"Soft, supple plum envelopes an oaky structure ...",Mountain Cuvée,87.0,19.0,California,Napa Valley,Napa,Virginie Boone,@vboone,Kirkland Signature 2011 Mountain Cuvée Caberne...,Cabernet Sauvignon,Kirkland Signature
4,"Slightly reduced, this wine offers a chalky, t...",NaN,87.0,34.0,California,Alexander Valley,Sonoma,Virginie Boone,@vboone,Louis M. Martini 2012 Cabernet Sauvignon (Alex...,Cabernet Sauvignon,Louis M. Martini


In [6]:
wine_US_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54504 entries, 0 to 54503
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   description            54504 non-null  object 
 1   designation            36908 non-null  object 
 2   points                 54504 non-null  float64
 3   price                  54265 non-null  float64
 4   province               54504 non-null  object 
 5   region_1               54226 non-null  object 
 6   region_2               50511 non-null  object 
 7   taster_name            37730 non-null  object 
 8   taster_twitter_handle  34741 non-null  object 
 9   title                  54504 non-null  object 
 10  variety                54504 non-null  object 
 11  winery                 54504 non-null  object 
dtypes: float64(2), object(10)
memory usage: 5.4+ MB


In [7]:
wine_US_1['points'] = wine_US_1['points'].astype('float64')
wine_US_1 = wine_US_1[(wine_US_1['price']!= 2013) & (wine_US_1['price'] != 750)]

In [10]:
wine_US_1 = wine_US_1[pd.isna(wine_US_1['taster_name']) == False]
wine_US_1 = wine_US_1[pd.isna(wine_US_1['description']) == False]
wine_US_1 = wine_US_1[pd.isna(wine_US_1['designation']) == False]
wine_US_1 = wine_US_1[pd.isna(wine_US_1['region_1']) == False]
wine_US_1 = wine_US_1[pd.isna(wine_US_1['region_2']) == False]
wine_US_1 = wine_US_1[pd.isna(wine_US_1['title']) == False]
sample_wine_US_1 = wine_US_1.sample(10000)
sample_wine_US_1.index = np.arange(0, sample_wine_US_1.shape[0])
wine_US_1_text = sample_wine_US_1.drop(['price', 'province', 'taster_name', 'taster_twitter_handle', 'variety', 'winery'], axis = 1)
wine_US_1_text.head()

,description,designation,points,region_1,region_2,title
0,Very concentrated purple flowers aromas of vio...,Bien Nacido X Block,91.0,Santa Maria Valley,Central Coast,Bonny Doon 2012 Bien Nacido X Block Syrah (San...
1,"This feral, smoky Merlot-based Bordeaux blend ...",Sonoma Cuvée,85.0,Sonoma County,Sonoma,St. Francis 2011 Sonoma Cuvée Red (Sonoma County)
2,"Merlot makes up half of this blend, with the b...",Reserve Cavalie,87.0,Horse Heaven Hills,Columbia Valley,Mercer 2012 Reserve Cavalie Red (Horse Heaven ...
3,The inaugural release from this négociant wine...,Vin de Table,91.0,Columbia Valley (WA),Columbia Valley,Les Trouvés 2013 Vin de Table Red (Columbia Va...
4,An arresting blend of Cabernet Sauvignon (62%)...,Carriage House DuBrul Vineyard,94.0,Yakima Valley,Columbia Valley,Côte Bonneville 2009 Carriage House DuBrul Vin...


In [11]:
X = wine_US_1_text.drop('points', axis = 1)
y = wine_US_1_text['points']
X_train, X_test, y_train, y_test = train_test_split(X,y)
X_train.head()

,description,designation,region_1,region_2,title
2749,"Harvested at a lip-smacking 42.5° brix, this s...",Late Harvest,Columbia Valley (WA),Columbia Valley,Chateau Ste. Michelle 1997 Late Harvest Sémill...
4938,"Broad aromas of Asian pear, kaffir lime and we...",Spanish Springs Vineyard,San Luis Obispo County,Central Coast,Stephen Ross 2016 Spanish Springs Vineyard Pin...
4396,High-toned red fruit wraps around jolts of bla...,R. Buoncristiani Vineyard,Russian River Valley,Sonoma,Matrix 2013 R. Buoncristiani Vineyard Pinot No...
8273,Lime peels and lemon pith mix it up with rain ...,Sanford & Benedict,Sta. Rita Hills,Central Coast,Sandhi 2013 Sanford & Benedict Chardonnay (Sta...
2272,"Out of Syncline's lineup of pure, varietal whi...",Boushey Vineyard,Yakima Valley,Columbia Valley,Syncline 2011 Boushey Vineyard Marsanne (Yakim...


#### Using SpaCY Word2Vec model here

In [12]:
# load pipeline object
nlp = pipeline("feature-extraction")

In [14]:
region1_pipe = [nlp(doc.encode().decode()) for doc in X_train['region_1']]

In [76]:
region_list = []
for i in range(len(region1_pipe)):
    region_list.append(np.mean(np.array(region1_pipe[i][0]), axis = 0))
region1_pipe_result = np.vstack(region_list)
region1_pipe_result.shape

(7500, 768)

In [81]:
description_pipe = [nlp(doc.encode().decode()) for doc in X_train['description']]
description_list = []
for i in range(len(region1_pipe)):
    description_list.append(np.mean(np.array(description_pipe[i][0]), axis = 0))
description_pipe_result = np.vstack(description_list)
description_pipe_result.shape

(7500, 768)

#### Due to the limitation of ram, we are only able to do description and region1 using BERT.

In [83]:
nlp2 = spacy.load("en_core_web_lg", disable=["tagger", "parser", "ner"])

In [84]:
# transform each sentence into vector and make the word2vec matrix
descript2vec = np.vstack([nlp2(doc.encode().decode()).vector for doc in X_train['description']])
design2vec = np.vstack([nlp2(doc.encode().decode()).vector for doc in X_train['designation']])
region12vec = np.vstack([nlp2(doc.encode().decode()).vector for doc in X_train['region_1']])
region22vec = np.vstack([nlp2(doc.encode().decode()).vector for doc in X_train['region_2']])
title2vec = np.vstack([nlp2(doc.encode().decode()).vector for doc in X_train['title']])

In [90]:
bow_model = CountVectorizer(ngram_range=(1, 3), min_df = 1, token_pattern=r"\b\w[\w’]+\b")

In [91]:
# use CountVectorizer to transform each text feature in order to combine them with word2vec matrix
descript_bow = bow_model.fit_transform(X_train['description']).toarray()
design_bow = bow_model.fit_transform(X_train['designation']).toarray()
region1_bow = bow_model.fit_transform(X_train['region_1']).toarray()
region2_bow = bow_model.fit_transform(X_train['region_2']).toarray()
title_bow = bow_model.fit_transform(X_train['title']).toarray()

In [86]:
X_bert_1 = np.concatenate((description_pipe_result, design2vec, region1_pipe_result, region22vec, title2vec), axis = 1)

In [87]:
X_bert_1.shape

(7500, 2436)

In [92]:
Ridge_bert_score = cross_val_score(Ridge(), X_bert_1, np.ravel(y_train), cv = 5)
print('The result of transfering description and retion_1 into vector by using bert is', np.mean(Ridge_bert_score))

The result of transfering description and retion_1 into vector by using bert is 0.5998747215841055


#### Adding description and region_1 preprocessed by BERT to the final model in Task 2.

In [95]:
X_train_bert = np.concatenate((description_pipe_result, descript2vec, descript_bow, design2vec, region1_pipe_result, region1_bow, region2_bow, title_bow), axis = 1)
Ridge_bert_score_6 = cross_val_score(Ridge(), X_train_bert, np.ravel(y_train), cv = 5)
print('The result of X_train_bert_score_6 is', np.mean(Ridge_bert_score_6))

The result of X_train_bert_score_6 is 0.6804836753232311


After adding the description and region preprossesed with BERT, we improve the score by approximately 0.03.